In [6]:
# Imports
import hashlib
import random
import sys
import os

def hash_transaction(transaction):
    return hashlib.sha256(transaction.encode('utf-8')).hexdigest()

def generate_sample_transactions(n=8):
    return [f"tx_{random.randint(10000, 99999)}_{i}" for i in range(n)]

transactions = generate_sample_transactions(8)
hashes = [hash_transaction(tx) for tx in transactions]

print("🟢 Transactions generated and hashed!")
for i, tx in enumerate(transactions):
    print(f"{i+1}. Transaction: {tx}\n   Hash: {hashes[i]}")

# ✅ Checkpoint 1: Transactions created and hashed!
with open('checkpoint.txt', 'w') as f:
    f.write('Checkpoint 1: Transactions created and hashed.')

print("✅ Checkpoint saved! You may continue with Merkle Tree construction.")

🟢 Transactions generated and hashed!
1. Transaction: tx_73134_0
   Hash: 2b3a687e8754f3e6b81047ec25680da55fcb71214d5f0ecda1399dab254381c7
2. Transaction: tx_31584_1
   Hash: 6fcfd002d5338a15e5648a6c2842527ba559e14642761b60d1c0122d8b1948ed
3. Transaction: tx_68660_2
   Hash: 26bd15aa45155bb436b0df7af26a668d6fb23536e792b65d7591460854eda848
4. Transaction: tx_63550_3
   Hash: cd9704636f065871764ce897ba714f27ef232f6ac0dd0c3fa272c15a7b246821
5. Transaction: tx_55442_4
   Hash: b9c2b631af642fc3886d34a1f6131b4252cab072bf4f3ad5de98801b4eb6e18f
6. Transaction: tx_37793_5
   Hash: daf804b885b9dbc944d617ce916f79719664b9ec7787110c9e7fd8bba2ceef1b
7. Transaction: tx_39381_6
   Hash: 2fbba3c562713814dc8a758a2f11f38a84a74d1edd6bf9e0fde21dc1e3ba8b99
8. Transaction: tx_59439_7
   Hash: eb58418483147cbaeffe1a07333aef4208623f992f221825208ca40f6bff9bf5
✅ Checkpoint saved! You may continue with Merkle Tree construction.


In [7]:
def merkle_tree(hashes):
    current_level = hashes.copy()
    tree_levels = [current_level]
    while len(current_level) > 1:
        next_level = []
        if len(current_level) % 2 != 0:
            current_level.append(current_level[-1])
        for i in range(0, len(current_level), 2):
            pair_hash = hashlib.sha256((current_level[i] + current_level[i+1]).encode('utf-8')).hexdigest()
            next_level.append(pair_hash)
        current_level = next_level
        tree_levels.append(current_level)
    return tree_levels

tree_levels = merkle_tree(hashes)
merkle_root = tree_levels[-1][0]

print(f"🌳 Merkle Tree constructed with {len(tree_levels)} levels!")
print(f"✨ Merkle Root: {merkle_root}")

for i, level in enumerate(tree_levels):
    print(f"Level {i+1} hashes:")
    for h in level:
        print(h)
    print()

with open('checkpoint.txt', 'a') as f:
    f.write('\nCheckpoint 2: Merkle Tree constructed & Merkle Root computed.')

print("✅ Checkpoint added! Ready for Merkle Proof implementation.")

🌳 Merkle Tree constructed with 4 levels!
✨ Merkle Root: de9e1fc4af7800e202cfd06ad51e9ecd6b34827aabb0f6ee128fed33a8ee857c
Level 1 hashes:
2b3a687e8754f3e6b81047ec25680da55fcb71214d5f0ecda1399dab254381c7
6fcfd002d5338a15e5648a6c2842527ba559e14642761b60d1c0122d8b1948ed
26bd15aa45155bb436b0df7af26a668d6fb23536e792b65d7591460854eda848
cd9704636f065871764ce897ba714f27ef232f6ac0dd0c3fa272c15a7b246821
b9c2b631af642fc3886d34a1f6131b4252cab072bf4f3ad5de98801b4eb6e18f
daf804b885b9dbc944d617ce916f79719664b9ec7787110c9e7fd8bba2ceef1b
2fbba3c562713814dc8a758a2f11f38a84a74d1edd6bf9e0fde21dc1e3ba8b99
eb58418483147cbaeffe1a07333aef4208623f992f221825208ca40f6bff9bf5

Level 2 hashes:
cb1f3c32de93efdbe5785f4e174d01b5cd8e7360c49d800ea89a8281df2cc343
80ff7d43d5807badd54d348d84f4d7cb63aea714311ea17445fb27fe6a0f9425
782ad8781ead666883e7ec2fca04aa0edf9b69f3bfa13b02176ef4c2ad48749c
246ec206da06db1a192fac0b9db29a47836b48758daf45677c9e7a06bff3212d

Level 3 hashes:
b5734132cfa9e7648c25178a9252a0d8636948418d59ba9ab

In [8]:
def merkle_proof(tree_levels, tx_index):
    proof = []
    index = tx_index
    for level in tree_levels[:-1]:
        sibling_index = index ^ 1
        if sibling_index < len(level):
            proof.append(level[sibling_index])
        index //= 2
    return proof

def verify_merkle_proof(tx_hash, proof, merkle_root, tx_index):
    computed_hash = tx_hash
    index = tx_index
    for sibling_hash in proof:
        if index % 2 == 0:
            computed_hash = hashlib.sha256((computed_hash + sibling_hash).encode('utf-8')).hexdigest()
        else:
            computed_hash = hashlib.sha256((sibling_hash + computed_hash).encode('utf-8')).hexdigest()
        index //= 2
    return computed_hash == merkle_root

# Verify transaction 3 (index=2)
tx_idx = 2
tx_hash = hashes[tx_idx]
proof = merkle_proof(tree_levels, tx_idx)
result = verify_merkle_proof(tx_hash, proof, merkle_root, tx_idx)

print(f"🔍 Merkle Proof for Transaction {tx_idx+1}:")
for h in proof:
    print(h)
print(f"\n🎯 Validity: {'✅ Transaction is part of the Merkle Tree!' if result else '❌ Transaction NOT in Merkle Tree!'}")

with open('checkpoint.txt', 'a') as f:
    f.write('\nCheckpoint 3: Merkle Proof verified for transaction index %d.' % tx_idx)

print("✅ Final checkpoint saved! Experiment complete.")

🔍 Merkle Proof for Transaction 3:
cd9704636f065871764ce897ba714f27ef232f6ac0dd0c3fa272c15a7b246821
cb1f3c32de93efdbe5785f4e174d01b5cd8e7360c49d800ea89a8281df2cc343
a0a0d7fc9b54bf5fd7d235feed43c14372a0138870e56e6b9d0af5010de700bb

🎯 Validity: ✅ Transaction is part of the Merkle Tree!
✅ Final checkpoint saved! Experiment complete.
